<!-- TABS -->
# Perform a vector search

In [ ]:
# <testing: MongoDB>
from superduperdb import superduper, Document
from superduperdb.backends.mongodb import Collection


db = superduper('mongomock://temp')

datas = []
for i in range(10):
    data = {
        "id": str(i),
        "my_key": str(i)
    }
    datas.append(data)

db.execute(Collection("documents").insert_many([Document(data) for data in datas]))
select = Collection("documents").find()

In [ ]:
# <testing: SQL>
from superduperdb import superduper, Document, Schema
from superduperdb.backends.ibis import Table, dtype

db = superduper('sqlite://')
table = Table(identifier="documents",
          schema=Schema(
              identifier="documents",
              fields={"id": dtype('str'), "my_key": dtype("str")}
          )
         )
db.add(table)


datas = []
for i in range(10):
    data = {
        "id": str(i),
        "my_key": str(i)
    }
    datas.append(data)

db.execute(table.insert([Document(data) for data in datas]))
table = db.load('table', "documents").to_query()
select = table.select("id", "my_key")

In [ ]:
# <testing: >
from superduperdb import ObjectModel, VectorIndex, Listener
from superduperdb.ext.numpy import array
import numpy as np

model = ObjectModel("test", object=lambda x: np.random.randn(16), datatype=array("float64", shape=(16, )))
db.add(
    VectorIndex(
        # Use a dynamic identifier based on the model's identifier
        identifier="vector_index",
        # Specify an indexing listener with MongoDB collection and model
        indexing_listener=Listener(
            select=select,  # MongoDB collection query
            key="my_key",  # Key for the documents
            model=model,
            predict_kwargs={"max_chunk_size": 16},
            identifier="vector",
        ),
    )
)
data = "10"

- **item** is the item which is to be encoded
- **my_key** is the attribute concerned during vector search, which will be transformed into a vector by the corresponding model.

In [ ]:
from superduperdb import Document

item = Document({'my_key': data})

## Vector search

Once we have this search target, we can execute a search as follows:

In [ ]:
# <tab: MongoDB>
from superduperdb.backends.mongodb import Collection

collection = Collection('documents')

select = collection.like(
    item, 
    vector_index="vector_index", 
    n=5
).find()

In [ ]:
# <tab: SQL>

# Table was created earlier, before preparing vector-search
table = db.load('table', 'documents')
select = table.like(
    item, 
    vector_index="vector_index", 
    n=5
).select("id", "my_key")

## Vector search with filters

In practical applications, we might aim to search for data that meets specific filters. Therefore, we can add filters, so that SuperDuperDB will only perform vector search within the set of data that meets the filter criteria.

In [ ]:
# <tab: MongoDB>
from superduperdb.backends.mongodb import Collection

collection = Collection('documents')

select = collection.find({"id": {"$in": ["1", "2"]}}).like(
    item,
    vector_index="vector_index",
    n=5
)

In [ ]:
# <tab: SQL>

# Table was created earlier, before preparing vector-search
table = db.load('table', 'documents')
select = table.select("id", "my_key").filter(table.id.isin(("1", "2")),).like(
    item,
    vector_index="vector_index",
    n=5
)

In [ ]:
# <testing: >
results = db.execute(select)
list(results)